# Download representative images of volcanos


Credit: this script derived from [Aaron Geller's blog post.](https://sites.northwestern.edu/researchcomputing/2021/11/19/downloading-satellite-images-made-easy/), including the use of functions based on his [EarthEngineToGeoTIFF repository.](https://github.com/ageller/EarthEngineToGeoTIFF)

In [ ]:
import ee
%pip install rasterio

import rasterio
from rasterio.plot import show as showRasterio

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
OUTPUT_IMAGE_DIR = DRIVE_MOUNT +"/My Drive/Full_landsat8_10km/"
PRIOR_OUTPUT_IMAGE_DIR = PROJECT_DIR +"Full_landsat8_10km_prior_year/"

TIFS_DIR = DRIVE_MOUNT+"/My Drive/tifs/"

lst = os.listdir(OUTPUT_IMAGE_DIR)
print(lst)

targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")


Mounted at /content/drive
[]


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=IhuUigsHuqywWg_QxzMmn6UpzcM1e9cF-NrgS6M3M-A&tc=4_YnE_FYZ_7unr-V6Jxelrqv9HJta7Q-6KmRuJDcEWQ&cc=575N9tAn_7fzht222xM35xKPACrNcMn3122TlN1CZBU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjzFLLGQQ5bY2Io5SO8oAQ9omf1pv4QOszM4x7BbDiA8_S4d284d9M

Successfully saved authorization token.


In [ ]:

import time

sleep_interval = 20
wait_limit = 900
download = True

def get_landsat8_image(lon, lat, sze_m, filename_base, 
                         dateMin = '2020-04-01', dateMax = '2020-04-30', 
                         temp_dir = "tifs",
                           output_dir = OUTPUT_IMAGE_DIR):
    '''    
    download an SARimage from the Sentinal S2 Surface Reflectance satellite, at the given coordinates
    
    lon : central longitude in degrees
    lat : central latitude in degrees
    sze_km : size of the edge of the box in km
    dateMin : minimum date to use for image search in year-month-day (e.g., 2020-08-01)
    dateMax : maximum date to use for image search in year-month-day (e.g., 2020-08-31)
    base_filename : base output filename for the GeoTIFF images. Will create 6 files for each bse
     For each of ASCENDING and DECENDING:
      - mean of VH channel
      - mean of VV channel
     mean of Ascending and descending:
      - VV channel
      - VH channel 
    
    https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA
    '''

    print(f"{lon} {lat} {filename_base} {dateMin} {dateMax} {temp_dir} {output_dir}")
    download = True
    if (download):
      # Calculate Area of Interest
      pt = ee.Geometry.Point(lon,lat)
          
      aoi = pt.buffer(distance=sze_m)

      # Load the LandSat data
      LS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
                               .filterBounds(aoi)\
                               .filterDate(dateMin,dateMax)\
                               .filter(ee.Filter.lt('CLOUD_COVER', 15))\
                               .mean()\
                               .set('SENSOR_ID', 'OLI_TIRS')

      # Bands to output
      #B1 	30 meters 	0.43 - 0.45 μm 	Coastal aerosol
      #B2 	30 meters 	0.45 - 0.51 μm 	Blue
      #B3 	30 meters 	0.53 - 0.59 μm 	Green
      #B4 	30 meters 	0.64 - 0.67 μm 	Red
      #B5 	30 meters 	0.85 - 0.88 μm 	Near infrared
      #B6 	30 meters 	1.57 - 1.65 μm 	Shortwave infrared 1
      #B7 	30 meters 	2.11 - 2.29 μm 	Shortwave infrared 2
      #B8 	15 meters 	0.52 - 0.90 μm 	Band 8 Panchromatic
      #B9 	15 meters 	1.36 - 1.38 μm 	Cirrus
      #B10 	30 meters 	10.60 - 11.19 μm Thermal infrared 1, resampled from 100m to 30m
      #B11 	30 meters 	11.50 - 12.51 μm Thermal infrared 2, resampled from 100m to 30m
      bands = ["B2","B3","B4","B5","B6","B7","B9","B10","B11"]

      task_list = []
      for selection in bands:
          #print(f"Task: {filename_base} {selection}")
          task = ee.batch.Export.image.toDrive(image=LS8.select(selection),
                                            description=filename_base+"_"+selection,
                                            scale=30,
                                            region=aoi,
                                            fileNamePrefix=filename_base+"_"+selection,
                                            folder=temp_dir,
                                            crs='EPSG:4326',
                                            fileFormat='GeoTIFF')

          task.start()
          task_list.append(task)
      
      loop_count= 0

      for task in task_list:
        while(task.active() and sleep_interval*loop_count < wait_limit):
           loop_count += 1
           time.sleep(sleep_interval)
           #print(f'slept {sleep_interval*loop_count}')
          
        status = task.status()

        if (status['state'] == 'COMPLETED' or status['state'] == 'READY'):
            print(f"Success: {task}")#print(status['destination_uris'])
        else:
            print (status['state'])
            if (status['state'] == "FAILED"):
               print(f"Failed: {task} {status['error_message']}")

      time.sleep(60) # sleep for final file to write

    # Group bands by output files
    bandsets = {"rgb" : ["B4","B3","B2"],
                "B5_B6_B7" : ["B5","B6","B7"],
                "B9_B10_B11" : ["B9","B10","B11"]}

    for bandset in bandsets.keys():
      # convert image to png
       f,ax = plt.subplots(figsize=(15,15))
       output = OUTPUT_IMAGE_DIR+filename_base+"_"+bandset+".png"
       for iband, band in enumerate(bandsets[bandset]):
         rast = rasterio.open(TIFS_DIR+filename_base+"_"+band+".tif")
         img = rast.read()
         if (iband == 0):
            nz, ny, nx = img.shape
            image = np.zeros((3,ny,nx))
            transform=rast.transform
         image[iband,:,:] = img[0,:,:]
         rast.close()

       showRasterio(image, ax = ax, transform=transform)    
       f.savefig(output, bbox_inches='tight')
       plt.close(f)

       print(f"wrote {output}")

    # clean up tifs left behind
    lst = os.listdir(TIFS_DIR)
   
    regex_pattern = filename_base + ".*.tif"
    for f in lst:
       if re.match(regex_pattern, f):
         print(f"removing {f}")
         os.remove(TIFS_DIR+"/"+f)


In [ ]:
target_size_m = 10000
pull_eruption_year = True
pull_prior_year = True

for i_eruption in range(2,3):#targets.shape[0]):
    volc = targets.loc[i_eruption]
    v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
   

    # adjust eruption date by 1 year
    erupt_start_date = date(volc["Start Year"].astype(int),
                            volc["Start Month"].astype(int),
                            volc["Start Day"].astype(int))
    erupt_end_date = date(volc["End Year"].astype(int),
                          volc["End Month"].astype(int),
                          volc["End Day"].astype(int))

    if (erupt_start_date >= date(2015,6,23)):
      print(v_name,erupt_start_date)
      pre_erupt_date = erupt_start_date - timedelta(weeks=6)

      if pull_eruption_year:
        # do prior to eruption too
        
        volc_file_name = f'{v_name}__{pre_erupt_date}_pre_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
        print(i_eruption, volc_file_name)
          
        try:
            _ =get_landsat8_image(volc["Longitude"], volc["Latitude"], target_size_m,
                            volc_file_name,
                                output_dir = OUTPUT_IMAGE_DIR,
                                dateMin = pre_erupt_date.isoformat(), dateMax=erupt_start_date.isoformat())
        except Exception as e:
            print(f'Failed to get data for {volc_file_name}')
            print(e)
      
      if pull_prior_year:
        prior_volc_file_name = f'{v_name}__{pre_erupt_date-timedelta(weeks=52)}_pre_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
  
        print(i_eruption, prior_volc_file_name)
          
        try:
            _ =get_landsat8_image(volc["Longitude"], volc["Latitude"], target_size_m,
                            volc_file_name,
                               output_dir = PRIOR_OUTPUT_IMAGE_DIR,
                               dateMin =  (pre_erupt_date-timedelta(weeks=52)).isoformat(), dateMax=(erupt_start_date-timedelta(weeks=52)).isoformat())
        except Exception as e:
            print(f'Failed to get data for prior year {prior_volc_file_name}')
            print(e)
      
      next_start_date = erupt_start_date
      next_end_date = erupt_start_date + timedelta(days=7)
      while (next_end_date <= erupt_end_date + timedelta(days=7) and next_end_date <= erupt_start_date + timedelta(days=187)):
          volc_file_name = f'{v_name}_{next_start_date}_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
          prior_volc_file_name = f'{v_name}_{next_start_date-timedelta(weeks=52)}_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'

          print(i_eruption, volc_file_name)
          if pull_eruption_year:
            try:
                _ =get_landsat8_image(volc["Longitude"], volc["Latitude"], 
                            target_size_m, volc_file_name,
                            output_dir = OUTPUT_IMAGE_DIR,
                            dateMin = next_start_date.isoformat(), 
                            dateMax=next_end_date.isoformat())
            except Exception as e:
              print(f'Failed to get data for {volc_file_name}:\n {e}')
          if pull_prior_year:
            try:
                _ =get_landsat8_image(volc["Longitude"], volc["Latitude"], 
                            target_size_m, prior_volc_file_name,
                            output_dir = PRIOR_OUTPUT_IMAGE_DIR,
                            dateMin = (next_start_date-timedelta(weeks=52)).isoformat(), 
                            dateMax=(next_end_date-timedelta(weeks=52)).isoformat())
            except Exception as e:
              print(f'Failed to get data for {volc_file_name}:\n {e}')
          next_start_date = next_start_date + timedelta(days=7)
          next_end_date = next_end_date + timedelta(days=7)
        

In [ ]:
lat = 12.702;
lon = -87.004;
dateMin = '2020-01-01';
dateMax = '2021-1-01';
sze_m = 5000;
filename_base = "San_Cristobal"

_ =get_landsat8_image(lon,lat,sze_m,filename_base,dateMin,dateMax)